In [3]:
#Libraries
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

In [2]:
# Open url
driver = webdriver.Chrome('chromedriver.exe')
url = f'http://data.un.org/Data.aspx?d=WDI&f=Indicator_Code%3aNY.GNP.PCAP.PP.CD'
driver.get(url)

In [4]:
itens_tabela=[]

#primeira página
# pega a tabela completa, codigo em html
tabela_completa_selenium = driver.find_element_by_xpath('//*[@id="divData"]/div[1]').get_attribute('innerHTML')
# transforma o resultado para BeatifulSoup
tabela_completa_bs = BeautifulSoup(tabela_completa_selenium)
#busca os elementos td
elementos_td = tabela_completa_bs.find_all('td')
#pega o texto dos itens, transforma em lista
itens_tabela.append([x.text for x in elementos_td])

In [5]:
#página 2 a 136
for page in range(0,135):
    # localiza e clica no botao de passar a pagina
    driver.find_element_by_xpath('//*[@id="linkNextB"]').click()
    # pega a tabela completa, codigo em html
    tabela_completa_selenium = driver.find_element_by_xpath('//*[@id="divData"]/div[1]').get_attribute('innerHTML')
    # transforma o resultado para BeatifulSoup
    tabela_completa_bs = BeautifulSoup(tabela_completa_selenium)
    #busca os elementos td
    elementos_td = tabela_completa_bs.find_all('td')
    #pega o texto dos itens, transforma em lista
    itens_tabela.append([x.text for x in elementos_td])
    time.sleep(2)

In [6]:
len(itens_tabela)

136

In [7]:
#transforma em array
tabela1 = np.array(itens_tabela).reshape(-1)

In [9]:
#reshaping
tabela1 = np.array(tabela1).reshape((int(len(tabela1)/4), 4))

In [12]:
#transforma em dataframe
df_tabela1 = pd.DataFrame(tabela1)
df_tabela1

,0,1,2,3
0,Afghanistan,2019,2190,
1,Afghanistan,2018,2100,
2,Afghanistan,2017,2090,
3,Afghanistan,2016,2000,
4,Afghanistan,2015,2110,
...,...,...,...,...
6795,Zambia,2001,1660,
6796,Zambia,2000,1570,
6797,Zambia,1999,1510,
6798,Zambia,1998,1440,


In [14]:
#última página
# pega a tabela completa, codigo em html
tabela_completa_selenium = driver.find_element_by_xpath('//*[@id="divData"]/div[1]').get_attribute('innerHTML')
# transforma o resultado para BeatifulSoup
tabela_completa_bs = BeautifulSoup(tabela_completa_selenium)
#busca os elementos td
elementos_td = tabela_completa_bs.find_all('td')
#pega o texto dos itens, transforma em lista e depois faz um reshape
itens_tabela = [x.text for x in elementos_td]
tabela1 = np.array(itens_tabela).reshape((int(len(itens_tabela)/4), 4))
#transforma em dataframe
df_tabela2 = pd.DataFrame(tabela1)

In [16]:
gni_per_capita = pd.concat([df_tabela1, df_tabela2], ignore_index=True)

In [17]:
gni_per_capita

,0,1,2,3
0,Afghanistan,2019,2190,
1,Afghanistan,2018,2100,
2,Afghanistan,2017,2090,
3,Afghanistan,2016,2000,
4,Afghanistan,2015,2110,
...,...,...,...,...
6832,Zimbabwe,1994,1870,
6833,Zimbabwe,1993,1710,
6834,Zimbabwe,1992,1680,
6835,Zimbabwe,1991,1850,


In [21]:
gni_per_capita.rename(columns={0:'Country', 1:'Year', 'Year':'GNI Per Capita'}, inplace=True)

In [23]:
gni_per_capita.drop(columns=[3], inplace=True)

In [26]:
gni_per_capita = gni_per_capita.astype({'Year': 'int', 'GNI Per Capita': 'float'})

In [27]:
gni_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6837 entries, 0 to 6836
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         6837 non-null   object 
 1   Year            6837 non-null   int32  
 2   GNI Per Capita  6837 non-null   float64
dtypes: float64(1), int32(1), object(1)
memory usage: 133.7+ KB


In [28]:
gni_per_capita = gni_per_capita.replace('United States', 'United States of America')

In [29]:
gni_per_capita.loc[30:90]

,Country,Year,GNI Per Capita
30,Albania,2000,3980.0
31,Albania,1999,3550.0
32,Albania,1998,3110.0
33,Albania,1997,2780.0
34,Albania,1996,3050.0
...,...,...,...
86,Albania,1994,2300.0
87,Albania,1993,2110.0
88,Albania,1992,1740.0
89,Albania,1991,1880.0


In [31]:
gni_per_capita.loc[gni_per_capita['GNI Per Capita'] < 1036, 'Economy Classification'] = 'Low income'
gni_per_capita.loc[gni_per_capita['GNI Per Capita'] >= 1036 and <4045, 'Economy Classification'] = 'Lower-middle income'
gni_per_capita.loc[gni_per_capita['GNI Per Capita'] >= 4046 and <12535, 'Economy Classification'] = 'Upper-middle income'
gni_per_capita.loc[gni_per_capita['GNI Per Capita'] >= 12535, 'Economy Classification'] = 'High income'

SyntaxError: invalid syntax (<ipython-input-31-ecf4e0f5741b>, line 2)

In [32]:
gni_per_capita.loc[gni_per_capita['GNI Per Capita'] < 1036, 'Economy Classification'] = 'Low income'

In [46]:
gni_per_capita.loc[gni_per_capita['GNI Per Capita'] >= 12535, 'Economy Classification'] = 'High income'

In [57]:
gni_per_capita.loc[(gni_per_capita['GNI Per Capita'] >= 1036) & (gni_per_capita['GNI Per Capita'] < 4045), 
                   'Economy Classification'] = 'Lower-middle income'

In [58]:
gni_per_capita.loc[(gni_per_capita['GNI Per Capita'] >= 4046) & (gni_per_capita['GNI Per Capita'] < 12535), 
                   'Economy Classification'] = 'Upper-middle income'

In [59]:
gni_per_capita.loc[6000:6100]

,Country,Year,GNI Per Capita,Economy Classification
6000,Switzerland,2006,48460.0,High income
6001,Switzerland,2005,43980.0,High income
6002,Switzerland,2004,41610.0,High income
6003,Switzerland,2003,40190.0,High income
6004,Switzerland,2002,38720.0,High income
...,...,...,...,...
6096,Thailand,2000,7160.0,Upper-middle income
6097,Thailand,1999,6700.0,Upper-middle income
6098,Thailand,1998,6330.0,Upper-middle income
6099,Thailand,1997,6930.0,Upper-middle income


In [61]:
gni_per_capita.to_excel("gni.xlsx", sheet_name='gni_per_capita')

In [62]:
gni_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6837 entries, 0 to 6836
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 6837 non-null   object 
 1   Year                    6837 non-null   int32  
 2   GNI Per Capita          6837 non-null   float64
 3   Economy Classification  6837 non-null   object 
dtypes: float64(1), int32(1), object(2)
memory usage: 187.1+ KB
